# Creating Config Files

This notebook generates config files that specify supernova specific priors 
and fitting arguments. Individual files are created for different supernova 
surveys. The config files are save in yaml format and values typically follow the 
following template:

    hsiao_x1:
      object_id:
        kwargs:
          bounds:
            t0:
              - ? # Typically t0 - 10
              - ? # Typically t0 + 10
            x1:
              - -.5
              - .5
          phase_range:
            - -20
            - 50
        priors:
          z: ?
          t0: ?

    sn91bg:
      object_id:
        kwargs:
          bounds:
            c:
              - 0
              - 1
            x1:
              - 0.65
              - 1.25
            t0:
              - ? # Typically t0 - 10
              - ? # Typically t0 + 10
          phase_range:
            - -20
            - 50
        priors:
          z: ?
          t0: ?

We note that values for some supernovae may have been manually altered in the
config files after manually inspecting their  light curve fits. For this reason,
an existing config file should not be overwritten on the the assumption that
the automatically generated and currently existing config files are the same.

#### Table of Contents:
1. <a href='#SDSS'>SDSS - Sako et al. 2018</a>: Generate a config file for the Sloan Digital Sky Survey
1. <a href='#DES'>DES - SN3yr</a>: Generate a config file for the Dark Energy Survey
1. <a href='#CSP'>CSP - DR3</a>: Generate a config file for the Carnegie Suipernova Project


In [ ]:
from pathlib import Path

import sfdmap
import yaml
from astropy.table import join
from sndata.des import sn3yr
from sndata.sdss import sako18
from sndata.csp import dr3
from sndata._utils import convert_to_jd

out_dir = Path('../config_files')
out_dir.mkdir(exist_ok=True, parents=True)
DUST_MAP = sfdmap.SFDMap('../phot_class/schlegel98_dust_map/')


In [ ]:
def create_config_dict(id_arr, t0_arr, z_arr, ra_arr, dec_arr, t0_range=10, dust=True):
    """Create a dictionary with fitting params and kwargs for our pipeline
    
    Args:
        id_add (ndarray): An array of object Ids
        t0_arr (ndarray): An array of t0 values for each object
        z_arr  (ndarray): An array of redshift values for each object
        t0_range (float): The + / - bounds when fitting t0
    """
    
    config_dict = {'hsiao_x1': {}, 'sn91bg': {}}
    for obj_id, t0, z, ra, dec in zip(id_arr, t0_arr, z_arr, ra_arr, dec_arr):
        config_dict['hsiao_x1'][obj_id] = {'priors': {}, 'kwargs': {}}
        config_dict['hsiao_x1'][obj_id]['priors'] = {'t0': t0}
        config_dict['hsiao_x1'][obj_id]['kwargs'] = {'bounds': {}, 'phase_range': {}}
        config_dict['hsiao_x1'][obj_id]['kwargs']['bounds'] = {'t0': [t0 - t0_range, t0 + t0_range], 'x1': [-.5, .5]}
        config_dict['hsiao_x1'][obj_id]['kwargs']['phase_range'] = [-20, 50]

        config_dict['sn91bg'][obj_id] = {'priors': {}, 'kwargs': {}}
        config_dict['sn91bg'][obj_id]['priors'] = {'t0': t0}
        config_dict['sn91bg'][obj_id]['kwargs'] = {'bounds': {}, 'phase_range': {}}
        config_dict['sn91bg'][obj_id]['kwargs']['bounds'] = {'t0': [t0 - t0_range, t0 + t0_range], 'c': [0, 1], 'x1': [0.65, 1.25]}
        config_dict['sn91bg'][obj_id]['kwargs']['phase_range'] = [-20, 50]
        
        if dust:
            mwebv = float(DUST_MAP.ebv(ra, dec))
            config_dict['hsiao_x1'][obj_id]['priors']['mwebv'] = mwebv
            config_dict['sn91bg'][obj_id]['priors']['mwebv'] = mwebv            
        
        if z >= 0:  # SDSS masks missing z values with -9
            config_dict['hsiao_x1'][obj_id]['priors']['z'] = z
            config_dict['sn91bg'][obj_id]['priors']['z'] = z

    return config_dict


def raise_path_exists(path):
    """Raise an error if a path exists
    
    Args:
        path (Path): The path to check
    """

    if path.exists():
        raise ValueError(
            'Existing files may have manually modified values.'
            ' Be careful not to overwrite them!'
        )


## SDSS - Sako et al. 2018 <a id='SDSS'></a>

For SDSS, we use values from the "master" table of Sako et al. 2018.

In [ ]:
sako18.download_module_data()
master = sako18.load_table('master').to_pandas()
master['JD'] = master['MJDatPeakrmag'] + 2400000.5
master.head()


We config dictionaries both with and without exctinction.

In [ ]:
sdss_config_noebv = create_config_dict(
    master['CID'],
    master['JD'], 
    master['zCMB'],
    master['RA'],
    master['DEC'],
    dust=False
)

sdss_config_extinction = create_config_dict(
    master['CID'],
    master['JD'], 
    master['zCMB'],
    master['RA'],
    master['DEC'],
    dust=True
)


Since not all targets have published redshift values, we also add bounds on the redshift so it can be fit where necessary.

In [ ]:
for config in (sdss_config_noebv, sdss_config_extinction):
    for model, model_config in config.items():
        for obj_id, obj_config in model_config.items():
            obj_config['kwargs']['bounds']['z'] = [0.05, 0.4]
        

Finally, we dump the dictionaries to file.

In [ ]:
sdss_noebv_path = out_dir / Path('sdss_config_noext.yml')
raise_path_exists(sdss_noebv_path)
with open(sdss_noebv_path, 'w') as ofile:
    yaml.dump(sdss_config_noebv, ofile)

sdss_ebv_path = out_dir / Path('sdss_config_ext.yml')
raise_path_exists(sdss_ebv_path)
with open(sdss_ebv_path, 'w') as ofile:
    yaml.dump(sdss_config_extinction, ofile)


## DES - SN3yr <a id='DES'></a>

In [ ]:
sn3yr.download_module_data()
des_fit_res = sn3yr.load_table('SALT2mu_DES+LOWZ_C11.FITRES').to_pandas()
des_fit_res['JD'] = des_fit_res['PKMJD'] + 2400000.5
des_fit_res.CIDint = des_fit_res.CIDint.apply(lambda x: str(x).zfill(8))
des_fit_res.head()


In [ ]:
des_config_noebv = create_config_dict(
    des_fit_res['CIDint'],
    des_fit_res['JD'], 
    des_fit_res['zCMB'],
    des_fit_res['RA'],
    des_fit_res['DECL'],
    dust=False
)

des_config_extinction = create_config_dict(
    des_fit_res['CIDint'],
    des_fit_res['JD'], 
    des_fit_res['zCMB'],
    des_fit_res['RA'],
    des_fit_res['DECL'],
    dust=True
)


In [ ]:
des_noebv_path = out_dir / Path('des_config_noext.yml')
raise_path_exists(des_noebv_path)
with open(des_noebv_path, 'w') as ofile:
    yaml.dump(des_config_noebv, ofile)

des_ebv_path = out_dir / Path('des_config_ext.yml')
raise_path_exists(des_ebv_path)
with open(des_ebv_path, 'w') as ofile:
    yaml.dump(des_config_extinction, ofile)


## CSP - DR3 <a id='CSP'></a>

In [ ]:
dr3.download_module_data()

# Load DR3 data for all objects with a published t0
dr3_data = join(dr3.load_table(1), dr3.load_table(3), keys='SN')
dr3_data = dr3_data[~dr3_data['T(Bmax)'].mask]

ra = []
dec = []
for obj_id in dr3_data['SN']:
    obj_data = dr3.get_data_for_id(obj_id)
    ra.append(obj_data.meta['ra'])
    dec.append(obj_data.meta['dec'])


In [ ]:
# We typecast to avoid errors with PyYaml
csp_config = create_config_dict(
    [str(sn) for sn in dr3_data['SN']],
    [float(t) for t in convert_to_jd(dr3_data['T(Bmax)'])], 
    [float(z) for z in dr3_data['z']],
    ra,
    dec,
    dust=True,
    t0_range=3
)

csp_ebv_path = out_dir / Path('csp_config_ext.yml')
raise_path_exists(csp_ebv_path)
with open(csp_ebv_path, 'w') as ofile:
    yaml.safe_dump(csp_config, ofile)
